# Create boundary conditions for Pb

June, 2019

In [2]:
import matplotlib.pyplot as plt
import netCDF4 as nc
import numpy as np
import datetime
import matplotlib
from mpl_toolkits.basemap import Basemap
import cmocean
import csv
import pandas as pd
import pickle
from matplotlib.patches import Polygon 
from scipy.signal import savgol_filter
import scipy as sc
from matplotlib.patches import ConnectionPatch
import gsw

%matplotlib notebook

Load extended domain

In [3]:
mesh       = nc.Dataset('/data/brogalla/old/meshmasks/ANHA12_mesh1.nc')
mesh_lon   = np.array(mesh.variables['nav_lon'])
mesh_lat   = np.array(mesh.variables['nav_lat'])
mesh_bathy = np.array(mesh.variables['tmask'][0,:,:,:])

bathy_masked = np.ma.masked_where((mesh_bathy > 0.1), mesh_bathy)
ocean_masked = np.ma.masked_where((mesh_bathy < 0.1), mesh_bathy)

In [4]:
m      = nc.Dataset('/data/brogalla/run_storage/spin-up0-low-case/ANHA12_EXH006_2002_monthly.nc', 'r')
mdepth = np.array(m.variables['deptht'])
mlons  = np.array(m.variables['nav_lon'])
mlats  = np.array(m.variables['nav_lat'])

Load Pb data

In [5]:
folder= '/ocean/brogalla/GEOTRACES/data/'

CAA = pd.read_csv(folder+'Pb-paper-data.csv')
CAA_lons     = CAA['station lon'].dropna().astype('float').values[1:]
CAA_lats     = CAA['Station lat'].dropna().astype('float').values[1:]
CAA_names    = CAA['Station names'].dropna().astype('str')
CAA_Pb_short = [CAA['CB3'].dropna().astype('float').values, \
                CAA['CB4'].dropna().astype('float').values, \
                CAA['LS2'].dropna().astype('float').values, \
                CAA['BB1'].dropna().astype('float').values]

dPb_BB1  = CAA['BB1'][np.isfinite(CAA['BB1'])].astype('float')
dPb_BB2  = CAA['BB2'][np.isfinite(CAA['BB2'])].astype('float')
dPb_BB3  = CAA['BB3'][np.isfinite(CAA['BB3'])].astype('float')
dPb_CB1  = CAA['CB1'][np.isfinite(CAA['CB1'])].astype('float')
dPb_CB2  = CAA['CB2'][np.isfinite(CAA['CB2'])].astype('float')
dPb_CB3  = CAA['CB3'][np.isfinite(CAA['CB3'])].astype('float')
dPb_CB4  = CAA['CB4'][np.isfinite(CAA['CB4'])].astype('float')
dPb_CAA1 = CAA['CAA1'][np.isfinite(CAA['CAA1'])].astype('float')
dPb_CAA2 = CAA['CAA2'][np.isfinite(CAA['CAA2'])].astype('float')
dPb_CAA3 = CAA['CAA3'][np.isfinite(CAA['CAA3'])].astype('float')
dPb_CAA4 = CAA['CAA4'][np.isfinite(CAA['CAA4'])].astype('float')
dPb_CAA5 = CAA['CAA5'][np.isfinite(CAA['CAA5'])].astype('float')
dPb_CAA6 = CAA['CAA6'][np.isfinite(CAA['CAA6'])].astype('float')
dPb_CAA7 = CAA['CAA7'][np.isfinite(CAA['CAA7'])].astype('float')
dPb_CAA8 = CAA['CAA8'][np.isfinite(CAA['CAA8'])].astype('float')
dPb_CAA9 = CAA['CAA9'][np.isfinite(CAA['CAA9'])].astype('float')

depth_BB1  = CAA['Depth.1'][np.isfinite(CAA['BB1'])].astype('float')
depth_BB2  = CAA['Depth.2'][np.isfinite(CAA['BB2'])].astype('float')
depth_BB3  = CAA['Depth.3'][np.isfinite(CAA['BB3'])].astype('float')
depth_CB1  = CAA['Depth.13'][np.isfinite(CAA['CB1'])].astype('float')
depth_CB2  = CAA['Depth.14'][np.isfinite(CAA['CB2'])].astype('float')
depth_CB3  = CAA['Depth.15'][np.isfinite(CAA['CB3'])].astype('float')
depth_CB4  = CAA['Depth.16'][np.isfinite(CAA['CB4'])].astype('float')
depth_CAA1 = CAA['Depth.4'][np.isfinite(CAA['CAA1'])].astype('float')
depth_CAA2 = CAA['Depth.5'][np.isfinite(CAA['CAA2'])].astype('float')
depth_CAA3 = CAA['Depth.6'][np.isfinite(CAA['CAA3'])].astype('float')
depth_CAA4 = CAA['Depth.7'][np.isfinite(CAA['CAA4'])].astype('float')
depth_CAA5 = CAA['Depth.8'][np.isfinite(CAA['CAA5'])].astype('float')
depth_CAA6 = CAA['Depth.9'][np.isfinite(CAA['CAA6'])].astype('float')
depth_CAA7 = CAA['Depth.10'][np.isfinite(CAA['CAA7'])].astype('float')
depth_CAA8 = CAA['Depth.11'][np.isfinite(CAA['CAA8'])].astype('float')
depth_CAA9 = CAA['Depth.12'][np.isfinite(CAA['CAA9'])].astype('float')

GEOTRACES cruise North Atlantic:

In [6]:
Pb_NA = nc.Dataset(folder+'GEOTRACES_IDP2017_v2_Discrete_Sample_Data_5024209a_1.nc')

NA_lats   = np.array(Pb_NA.variables['latitude'])
NA_lons   = np.array(Pb_NA.variables['longitude'])
NA_depths = np.array(Pb_NA.variables['var2'])
NA_time   = np.array(Pb_NA.variables['date_time'])
Pb_NA     = np.array(Pb_NA.variables['var5']) # in pmol/kg

In [7]:
condNA           = np.logical_or( Pb_NA > 1e5, Pb_NA < -1e5)
masked_Pb_NA     = np.ma.masked_where(condNA, Pb_NA)
masked_depths_NA = np.ma.masked_where(condNA, NA_depths)

Pb from Arctic Ocean:

In [8]:
df = pd.read_csv(folder+'Pb_rob_rember.csv', parse_dates=['DATE']).dropna()
AO_depths = np.array(df['CTDDEPTH'])
AO_lats   = np.array(df['LATITUDE'])
AO_lons   = np.array(df['LONGITUDE'])
AO_dPb    = np.array(df['Pb_(pMol)'])

Boundary coordinates:

In [9]:
imin, imax = 160, 800
jmin, jmax = 1100, 2050

# Northern boundary

In [10]:
ANHA12lat_CB = mesh_lat[jmax,imin+42:imax-1]
Pb_CB        = np.empty_like(mesh_bathy[:,jmax,imin+42:imax-1])
yy_CB, zz_CB = np.meshgrid(ANHA12lat_CB, mdepth)

In [11]:
Pb_CB[zz_CB < 20]                                   = 5 
Pb_CB[(zz_CB > 20) & (zz_CB < 50)]                  = 4
Pb_CB[(zz_CB < 50) & (yy_CB < 72.3)]                = 3 
Pb_CB[(zz_CB > 50) & (zz_CB < 100) & (yy_CB < 74)]  = 12
Pb_CB[(zz_CB > 50) & (zz_CB < 100) & (yy_CB > 74)]  = 8  
Pb_CB[(zz_CB > 100) & (zz_CB < 400)]                = 6
Pb_CB[(zz_CB > 400) & (zz_CB < 800)]                = 5 
Pb_CB[(zz_CB > 300) & (zz_CB < 700) & (yy_CB > 75)] = 5
Pb_CB[(zz_CB > 250) & (zz_CB < 700) & (yy_CB > 77)] = 4
Pb_CB[(zz_CB > 700) & (zz_CB < 800) & (yy_CB > 75)] = 3 
Pb_CB[(zz_CB > 800) & (zz_CB < 2000)]               = 4 
Pb_CB[zz_CB > 2000]                                 = 3

In [12]:
smoothed_Pb_CB1 = np.empty_like(Pb_CB)
smoothed_Pb_CB = np.empty_like(Pb_CB)

# Smooth vertically (along depth)
for i in range(0, len(Pb_CB[0,:])):
    smoothed_Pb_CB1[:,i] = savgol_filter(Pb_CB[:,i], 15, 3)

# Smooth horizontally along cross section
for depth in range(0, len(Pb_CB[:,0])):
    smoothed_Pb_CB[depth,:] = savgol_filter(smoothed_Pb_CB1[depth,:], 31, 3)

Variable to save to file:

In [13]:
rimwidthN          = 10
dPb_North          = np.empty((1,50,1,yy_CB.shape[1]*rimwidthN))
dPb_North[0,:,0,:] = np.tile(smoothed_Pb_CB, rimwidthN)

In [14]:
print('Northern boundary condition shape: ', dPb_North.shape)

Northern boundary condition shape:  (1, 50, 1, 5970)


# Eastern boundary

Arctic Ocean section:

In [15]:
ANHA12lon_AO = mesh_lon[1600:jmax,799]
Pb_AO        = np.empty_like(mesh_bathy[:,1600:jmax,799])
xx_AO, zz_AO = np.meshgrid(ANHA12lon_AO, mdepth)

In [16]:
Pb_AO.fill(3) 
Pb_AO[zz_AO < 50]                     = 5 
Pb_AO[(zz_AO > 50)  & (zz_AO < 100)]  = 8 
Pb_AO[(zz_AO > 100) & (zz_AO < 400)]  = 6 
Pb_AO[(zz_AO > 400) & (zz_AO < 800)]  = 3 
Pb_AO[(zz_AO > 800) & (zz_AO < 2000)] = 4
Pb_AO[zz_AO > 2000]                   = 3

In [17]:
smoothed_Pb_AO1 = np.empty_like(Pb_AO)
smoothed_Pb_AO = np.empty_like(Pb_AO)

# Smooth vertically (along depth)
for i in range(0, len(Pb_AO[0,:])):
    smoothed_Pb_AO1[:,i] = savgol_filter(Pb_AO[:,i], 15, 3)

# Smooth horizontally along cross section
for depth in range(0, len(Pb_AO[:,0])):
    smoothed_Pb_AO[depth,:] = savgol_filter(smoothed_Pb_AO1[depth,:], 31, 3)

Greenland section:

In [18]:
ANHA12lon_GR = mesh_lon[jmin:1290,799]
Pb_GR        = np.empty_like(mesh_bathy[:,jmin:1290,799])
yy_GR, zz_GR = np.meshgrid(ANHA12lon_GR, mdepth)

In [19]:
Pb_GR.fill(20) 
Pb_GR[zz_GR < 50]                                                        = 16 
Pb_GR[(zz_GR > 350) & (zz_GR < 1500) & (yy_GR < 63.5) & (yy_GR > 61.5)]  = 23 
Pb_GR[(zz_GR > 350) & (zz_GR < 1500) & (yy_GR < 64) & (yy_GR < 61)]      = 18 
Pb_GR[zz_GR > 2000]                                                      = 10

In [20]:
smoothed_Pb_GR1 = np.empty_like(Pb_GR)
smoothed_Pb_GR = np.empty_like(Pb_GR)

# Smooth vertically (along depth)
for i in range(0, len(Pb_GR[0,:])):
    smoothed_Pb_GR1[:,i] = savgol_filter(Pb_GR[:,i], 15, 3)

# Smooth horizontally along cross section
for depth in range(0, len(Pb_GR[:,0])):
    smoothed_Pb_GR[depth,:] = savgol_filter(smoothed_Pb_GR1[depth,:], 31, 3)

In [21]:
filtered_Pb_GR = sc.ndimage.filters.gaussian_filter(smoothed_Pb_GR, 1.8, order=0, mode='wrap')

Combined boundary condition:

In [22]:
filtered_Pb_E = np.empty_like(mesh_bathy[:,jmin+2:jmax-1,799])
filtered_Pb_E.fill(3) 
filtered_Pb_E[:,(1600-jmin-4):-1] = smoothed_Pb_AO
filtered_Pb_E[:,0:(1290-jmin)]    = filtered_Pb_GR

In [23]:
xx_E, zz_E  = np.meshgrid(mesh_lon[jmin+2:jmax-1,799], mdepth) 

In [24]:
rimwidthE = 20
dPb_East  = np.empty((1,50,1,xx_E.shape[1]*rimwidthE))
dPb_East[0,:,0,:]  = np.tile(filtered_Pb_E , rimwidthE) 

In [25]:
print('Eastern boundary condition shape: ', dPb_East.shape)

Eastern boundary condition shape:  (1, 50, 1, 18940)


# Southern boundary

In [26]:
ANHA12lon_BB = mesh_lon[jmin,imin+42:imax-1]
xx, zz = np.meshgrid(ANHA12lon_BB, mdepth) 

In [29]:
Pb_BB = np.empty_like(mesh_bathy[:,jmin,imin+42:imax-1])

Pb_BB.fill(20)
Pb_BB[zz < 20]                               = 12
Pb_BB[(zz < 20)  & (xx > -50)]               = 16
Pb_BB[(zz < 100) & (zz > 20)]                = 18
Pb_BB[(zz < 100) & (zz > 20) & (xx < -50)]   = 14 
Pb_BB[(zz > 350) & (zz < 1500)]              = 33
Pb_BB[(zz > 350) & (zz < 1500) & (xx < -41)] = 23 
Pb_BB[(zz > 350) & (zz < 1500) & (xx > -36)] = 23 
Pb_BB[zz > 2500]                             = 10

In [30]:
smoothed_Pb_BB1 = np.empty_like(Pb_BB)
smoothed_Pb_BB = np.empty_like(Pb_BB)

# Smooth vertically (along depth)
for i in range(0, len(Pb_BB[0,:])):
    smoothed_Pb_BB1[:,i] = savgol_filter(Pb_BB[:,i], 21, 3)

# Smooth horizontally along cross section
for depth in range(0, len(Pb_BB[:,0])):
    smoothed_Pb_BB[depth,:] = savgol_filter(smoothed_Pb_BB1[depth,:], 81, 3)

In [31]:
filtered_Pb_BB = sc.ndimage.filters.gaussian_filter(smoothed_Pb_BB, 1.8, order=0, mode='wrap')

In [32]:
rimwidthS          = 10
dPb_South          = np.empty((1,50,1,xx.shape[1]*rimwidthS))
dPb_South[0,:,0,:] = np.tile(filtered_Pb_BB, rimwidthS) 

In [33]:
print('Southern boundary condition shape: ', dPb_South.shape)

Southern boundary condition shape:  (1, 50, 1, 5970)


# Write NetCDF BC files:

In [34]:
# Check whether any of the values are zero
print(np.count_nonzero(dPb_South==0), np.count_nonzero(dPb_East==0), np.count_nonzero(dPb_North==0))

0 0 0


In [37]:
# Setup the new NetCDF file:
ncd = nc.Dataset('/ocean/brogalla/GEOTRACES/data/Pb_OBC-20200403.nc', 'w', zlib=True)
ncd.createDimension('x1',yy_CB.shape[1]*rimwidthN)
ncd.createDimension('x2',xx.shape[1]*rimwidthS)
ncd.createDimension('x3',xx_E.shape[1]*rimwidthE)
ncd.createDimension('y',1)
ncd.createDimension('deptht',50)
ncd.createDimension('time_counter',None)

<class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time_counter', size = 0

In [38]:
#Northern BC: Canada Basin
dpb_N = ncd.createVariable('dPb_N', 'float64', ('time_counter','deptht','y','x1'))
dpb_N.units       = 'pmol/kg'
dpb_N.long_name   = 'Dissolved Pb concentration, northern boundary'  
dpb_N.coordinates = 'nav_lon nav_lat deptht time_counter'
dpb_N[:,:,:,:]    = dPb_North*10**(-12)

#Southern BC: Baffin Bay / Labrador Sea
dpb_S = ncd.createVariable('dPb_S', 'float64', ('time_counter','deptht','y','x2'))
dpb_S.units       = 'pmol/kg'
dpb_S.long_name   = 'Dissolved Pb concentration, southern boundary'  
dpb_S.coordinates = 'nav_lon nav_lat deptht time_counter'
dpb_S[:,:,:,:]    = dPb_South*10**(-12)

#Eastern BC: East Greenland and Arctic Ocean
dpb_E = ncd.createVariable('dPb_E', 'float64', ('time_counter','deptht','y','x3'))
dpb_E.units       = 'pmol/kg'
dpb_E.long_name   = 'Dissolved Pb concentration, eastern boundary'  
dpb_E.coordinates = 'nav_lon nav_lat deptht time_counter'
dpb_E[:,:,:,:]    = dPb_East*10**(-12)

In [39]:
ncd.close()